In [66]:
import pandas as pd
import math
import numpy as np
import itertools

In [76]:
#statsDF - initial stats dataframe
#gamesDF - initial games dataframe
#newGamesDF - formatted games dataframe with game date as index and rest of record in one row
#finalStats - formatted stats dataframe with player's name as the index
#mainDF: The dataframe where the main analysis will take place.

In [77]:
stats = '/home/acer/github/nba/stats.csv'
games = '/home/acer/github/nba/games.csv'

In [69]:
#Dump the stats dataframe into a csv and clean it up
cols = ['PLAYER', 'MIN', 'FGM','FGA', 'FG%', '3PM', '3PA','3P%','FTM','FTA','FT%','OREB','DREB','REB','AST','TOV','STL','BLK','PF','PTS','+/-']
statsDF = pd.read_csv(stats, header= None, names=cols, index_col=False, error_bad_lines=False, delim_whitespace=True)
statsDF[cols] = statsDF[cols].replace({'\$': '', ',': ''}, regex=True)
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace('.','')
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace('-','')
statsDF['PLAYER'] = statsDF['PLAYER'].str.replace("'",'')
statsDF['MIN'] = statsDF['MIN'].str.replace("'",'')
statsDF['MIN'] = statsDF['MIN'].replace({'\$': '', '-': ''}, regex=True)
#Drop the first line for the team name
if statsDF.iloc[0][0] == 'GLOSSARY':
    statsDF.drop([0,1], inplace=True)
else:
    statsDF.drop(0, inplace=True)

In [70]:
#The nba.com site changed its formatting so this is to remove non-existent rows

new_format_mask = 0
for index, row in statsDF.iterrows():
    if new_format_mask == 1:
        statsDF.drop(index, inplace=True)
        if row.iloc[0] == 'Totals:' and math.isnan(row.iloc[1]) == True:
            new_format_mask = 0 
        else:
            continue
    else:
        if row.iloc[0] == 'PLAYER' and row.iloc[1] != "MIN":
            new_format_mask += 1
        else:
            continue

for index, row in statsDF.iterrows():
    if row.iloc[0][3:4] == ":":
        statsDF.drop(index, inplace=True)

    

In [71]:
statsDF = statsDF[statsDF.PLAYER != 'PLAYER']
statsDF = statsDF[statsDF.PLAYER != 'Totals:']
statsDF = statsDF[statsDF.MIN != 'PLAYERS']
statsDF = statsDF.reset_index(drop=True)

In [72]:
#This will delete the second time a team is named. 
for index, row in statsDF.iterrows():
    print(row.iloc[1])
    try: 
        if row.iloc[1].isalpha() and statsDF.iloc[index + 1][1].isalpha() or row.iloc[1] == "76ers":
            statsDF.drop(index, inplace=True)
            statsDF = statsDF.reset_index(drop=True)
            break
    except:
        continue

In [73]:
#Setting up the index on the main stats dataframe
mainDFIndex = []
for index, row in statsDF.iterrows():
    if row.iloc[0] == 'Nene':
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
    elif row.iloc[0].isalpha() == True and row.iloc[1].isalpha() == True:
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
print(mainDFIndex)

[]


In [74]:
#Setup dicts from the DF
statsDF_Dict = statsDF.T.to_dict().values()

In [75]:
statsDF

,PLAYER,MIN,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,...,OREB,DREB,REB,AST,TOV,STL,BLK,PF,PTS,+/-


In [44]:
dict_list = []
#Iterate through the statsDF and only return lines with stats or lines where players sat
for index, i in enumerate(statsDF_Dict):
    if index == 1 or index % 2 != 0:
        dict_list.append(i)
    

In [45]:
finalStats = pd.DataFrame(dict_list)
finalStats.index = mainDFIndex
finalStats.drop(['+/-'], axis=1, inplace=True)
finalStats = finalStats.rename(columns={'PLAYER':'MIN','MIN':'FGM', 'FGM':'FGA', 'FGA':'FG%', 'FG%':'3PM', '3PM':'3PA', '3PA':'3P%', '3P%':'FTM', 'FTM':'FTA', 'FTA':'FT%', 'FT%':'OREB', 'OREB':'DREB', 'DREB':'REB', 'REB':'AST', 'AST':'TOV', 'TOV':'BLK', 'BLK':'PF', 'PF':'PTS', 'PTS':'+/-'})
#This will replace the nan's with DNP
finalStats = finalStats.fillna('NA')
finalStats['Player'] = finalStats.index

In [46]:
finalStats.head()

,FTM,3P%,3PA,TOV,PF,REB,3PM,FG%,FGA,OREB,...,FTA,FGM,DREB,PTS,MIN,+/-,AST,STL,BLK,Player
Wesley Johnson,0,0.0,1,0,1,3,0,0.0,1,0,...,0,0,3,0,18:25,4,1,0,2,Wesley Johnson
Blake Griffin,6,25.0,8,3,0,4,2,33.3,27,2,...,7,9,2,26,38:29,-13,6,0,1,Blake Griffin
DeAndre Jordan,0,0.0,0,1,3,14,0,85.7,7,2,...,1,6,12,12,37:06,-5,0,0,2,DeAndre Jordan
Sindarius Thornwell,2,50.0,2,2,5,2,1,40.0,5,2,...,2,2,0,7,31:52,-9,2,0,2,Sindarius Thornwell
Austin Rivers,0,55.6,9,2,4,3,5,38.9,18,1,...,0,7,2,19,34:21,-3,2,0,2,Austin Rivers


In [47]:
#GAMES CELL: This prepares the games section into a dataframe
#Passing in csv file and getting rid of the unused rows
gamesDF = pd.read_csv(games, header= None, names=['a','b','c','d','e','f','g','h','i'], index_col=False, error_bad_lines=False, delim_whitespace=True)
gamesDF.columns = [col.replace(',', '') for col in gamesDF.columns]
#if you want to operate on multiple columns, put them in a list like so:
games_cols = ['a','b','c','d','e','f','g','h','i']
# pass them to df.replace(), specifying each char and it's replacement:
gamesDF[games_cols] = gamesDF[games_cols].replace({'\$': '', ',': ''}, regex=True)
# This will add the winner to column b of each 26th row
for index, row in gamesDF.iterrows():
    if index % 26 == 0 or index == 0:
        if int(gamesDF['a'].iloc[index+4]) > int(gamesDF['a'].iloc[index + 8]):
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 1]
        else:
            gamesDF['b'].iloc[index] = gamesDF['a'].iloc[index + 5]
    else:
        pass

In [48]:
#This cell will arrange the gamesDF into the actual formatted games dataframe, newGamesDF
#The next lines will pull the data from the cells in the gamesDF and into the newGamesDF dataframe
gamesData = []
games = 0
for index, row in gamesDF.iterrows():
    if index == 0 or index % 26 == 0:
        gamesData.append(gamesDF.iloc[index+11,0])
        gamesData.append(gamesDF.iloc[index+12,0])
        gamesData.append(gamesDF.iloc[index,1])
        gamesData.append(gamesDF.iloc[index+11,1])
        gamesData.append(gamesDF.iloc[index+11,2])
        gamesData.append(gamesDF.iloc[index+11,3])
        gamesData.append(gamesDF.iloc[index+11,4])
        gamesData.append(gamesDF.iloc[index+12,1])
        gamesData.append(gamesDF.iloc[index+12,2])
        gamesData.append(gamesDF.iloc[index+12,3])
        gamesData.append(gamesDF.iloc[index+12,4])
        gamesData.append(gamesDF.iloc[index+8,0])
        gamesData.append(gamesDF.iloc[index+4,0])
        gamesData.append(gamesDF.iloc[index+22,2])
        gamesData.append(gamesDF.iloc[index+22,4])
        gamesData.append(gamesDF.iloc[index+22,6])
        games += 1
        
gamesData = np.array(gamesData)
gamesData = gamesData.reshape(games,16)
#Create an empty dataframe, based on the games date as the index
newGamesDF = pd.DataFrame(data = gamesData, columns=['Home','Away','W/L','1st Qtr H','2nd Qtr H','3rd Qtr H','4th Qtr H','1st Qtr A','2nd Qtr A','3rd Qtr A','4th Qtr A','Total H','Total A', 'Ref1','Ref2','Ref3'])
newGamesDF

,Home,Away,W/L,1st Qtr H,2nd Qtr H,3rd Qtr H,4th Qtr H,1st Qtr A,2nd Qtr A,3rd Qtr A,4th Qtr A,Total H,Total A,Ref1,Ref2,Ref3
0,LAC,NOP,NEW,26,32,24,21,31,32,25,23,111,103,"Guthrie""""","Cutler""""",Smith


In [49]:
for index,row in finalStats.iterrows():
    finalStats['1st Qtr H'] = newGamesDF['1st Qtr H'][0]
    finalStats['2nd Qtr H'] = newGamesDF['2nd Qtr H'][0]
    finalStats['3rd Qtr H'] = newGamesDF['3rd Qtr H'][0]
    finalStats['4th Qtr H'] = newGamesDF['4th Qtr H'][0]
    finalStats['1st Qtr A'] = newGamesDF['1st Qtr A'][0]
    finalStats['2nd Qtr A'] = newGamesDF['2nd Qtr A'][0]
    finalStats['3rd Qtr A'] = newGamesDF['3rd Qtr A'][0]
    finalStats['4th Qtr A'] = newGamesDF['4th Qtr A'][0]
    finalStats['Total H'] = newGamesDF['Total H'][0]
    finalStats['Total A'] = newGamesDF['Total A'][0]
    finalStats['Ref1'] = newGamesDF['Ref1'][0]
    finalStats['Ref2'] = newGamesDF['Ref2'][0]
    finalStats['Ref3'] = newGamesDF['Ref3'][0]
    finalStats['W/L'] = newGamesDF['W/L'][0]
    finalStats['Home'] = newGamesDF['Home'][0]
    finalStats['Away'] = newGamesDF['Away'][0]
    

In [50]:
#This function will calculate the projected fantasy points per game
def fantasy_points(row):
    try:
        score = (int(row['3PM']) * 3) + (int(row['FGM']) * 2) + (int(row['REB']) * 1) + (int(row['AST']) * 1.5) + (int(row['BLK']) * 3) + (int(row['STL']) * 3) + (int(row['TOV']) * -1)
        return score
    except:
        pass

finalStats['Fantasy Score'] = finalStats.apply(fantasy_points, axis=1)
finalStats = finalStats.fillna('NA')

In [51]:
finalStats.index

Index(['Wesley Johnson', 'Blake Griffin', 'DeAndre Jordan',
       'Sindarius Thornwell', 'Austin Rivers', 'Lou Williams', 'Jawun Evans',
       'Willie Reed', 'Montrezl Harrell', 'Brice Johnson', 'Sam Dekker',
       'Danilo Gallinari', 'CJ Williams', 'Dante Cunningham', 'Anthony Davis',
       'DeMarcus Cousins', 'ETwaun Moore', 'Jrue Holiday', 'Jameer Nelson',
       'Darius Miller', 'Cheick Diallo', 'Ian Clark', 'Tony Allen',
       'Charles Cooke', 'Jalen Jones', 'Rajon Rondo'],
      dtype='object')